# hotpot_qa

In [3]:
from datasets import load_dataset

dataset_name:str = "ambig_qa"
dataset = load_dataset("json", data_files="/Users/ariete/Projects/self-improve/output/inference/ambig_qa/react_1000_temperature_0_top-p_1.jsonl", split="train")
if dataset_name == "trivia_qa":
    dataset = dataset.map(lambda x: {"answer": x["answer"]["normalized_aliases"]})
print(dataset, dataset[0]["answer"])

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['question', 'answer', 'prediction'],
    num_rows: 1000
}) ['Tony Goldwyn', 'Goldwyn']


In [6]:
import numpy as np
from agent.utils.qa import em_f1_score, normalize_answer, multi_ref_score
em_scores = []
f1_scores = []
precision_scores = []
recall_scores = []
for idx, sample in enumerate(dataset):
        

    cur_em = []
    cur_f1 = []
    cur_precesion = []
    cur_recall = []
    predictions = [sample["prediction"]]
    predictions = [prediction.split("FINAL ANSWER:")[-1].strip() for prediction in predictions]

    while len(predictions) < 4:
        predictions.append(predictions[-1])
    
    sample["answer"] = [normalize_answer(answer) for answer in sample["answer"]]
    # sample["answer"] = normalize_answer(sample["answer"])
    predictions = [normalize_answer(prediction) for prediction in predictions]

    for idx, prediction in enumerate(predictions):

        em, f1, precision, recall = multi_ref_score(prediction, sample['answer'])

        cur_em.append(em)
        cur_f1.append(f1)
        cur_precesion.append(precision)
        cur_recall.append(recall)
        
        # # # early stop
        stop = (predictions[idx] == predictions[idx - 1])

        if em or stop:
            cur_em.extend([em] * (4 - idx - 1))
            cur_f1.extend([f1] * (4 - idx - 1))
            cur_precesion.extend([precision] * (4 - idx - 1))
            cur_recall.extend([recall] * (4 - idx - 1))
            break
    

    em_scores.append(cur_em)
    f1_scores.append(cur_f1)
    precision_scores.append(cur_precesion)
    recall_scores.append(cur_recall)
    

# output mean of each column of scores
em_means = np.array(em_scores).mean(axis=0)
em_means = np.round(em_means* 100, decimals=1)

f1_means = np.array(f1_scores).mean(axis=0)
f1_means = np.round(f1_means* 100, decimals=1)

precision_means = np.array(precision_scores).mean(axis=0)
precision_means = np.round(precision_means* 100, decimals=1)

recall_means = np.array(recall_scores).mean(axis=0)
recall_means = np.round(recall_means* 100, decimals=1)

print(f"em_means: {em_means}\n", f"f1_means: {f1_means}\n", f"precision_means: {precision_means}\n", f"recall_means: {recall_means}\n")


em_means: [44.4 44.4 44.4 44.4]
 f1_means: [58.3 58.3 58.3 58.3]
 precision_means: [58.8 58.8 58.8 58.8]
 recall_means: [61.6 61.6 61.6 61.6]

